In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
from matplotlib import pyplot as plt
import matplotlib as mpl
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from langdetect import detect

In [2]:
# https://matplotlib.org/stable/gallery/text_labels_and_annotations/fonts_demo.html
# font for displaying confusion matrix
font = {'family': 'sans-serif',
        #'color':  'darkred',
        'weight': 'heavy',
        'size': 6,
        }

In [5]:
sdg_names = pd.read_excel("Digital Science SDG training set searches.xlsx")
sdg_names = sdg_names.drop([0,1,2], axis=0)
sdg_names = sdg_names.set_axis(["sdg", "sdg_name", "sdg_definition"],axis=1, copy=False)
sdg_names.sdg_name.tolist()

['No Poverty',
 'Zero Hunger',
 'Good Health and Well Being',
 'Quality Education',
 'Gender Equality',
 'Clean Water and Sanitation',
 'Affordable and Clean Energy',
 'Decent Work and Economic Growth',
 'Industry, Innovation and Infrastructure',
 'Reduced Inequalities',
 'Sustainable Cities and Communities',
 'Responsible Consumption and Production',
 'Climate Action',
 'Life Below Water',
 'Life on Land',
 'Peace, Justice and Strong Institutions',
 'Partnerships for the Goals']

### get data ready
 - read data
 - format the columns and column names 
 - add language detection
 - filter by agreement and language

 - language detection takes long time, on the full SDG dataset, it takes neara 3 minutes
 - may not want to run it unless really needed, or run it after filtering by other dimensions

In [3]:
text_df = pd.read_csv("/Users/yingli/Downloads/osdg-community-data-v2023-01-01.csv", sep = "\t", quotechar='"')
col_names = text_df.columns.values[0].split('\t')
text_df[col_names] = text_df[text_df.columns.values[0]].apply(lambda x: pd.Series(str(x).split("\t")))
text_df.drop(text_df.columns.values[0],axis = 1, inplace=True)
text_df = text_df.astype({'sdg':int, 'labels_negative': int, 'labels_positive':int, 'agreement': float}, copy=True)
text_df = text_df.query("agreement > 0.5 and (labels_positive - labels_negative) > 2")
#text_df["lang"] = text_df["text"].apply(lambda x: detect(x))
#text_df = text_df.query("lang == 'en'")# language detection cost extra time, do it after other filterings to minimize cost
text_df.reset_index(inplace=True)

### study term frequencies

In [4]:
docs = text_df.text
count_vectorizer = CountVectorizer() # default is unigram, no stop word removal
count_vectorizer.fit(docs) 
len(count_vectorizer.vocabulary_)

45738

In [5]:
count_vectorizer.vocabulary_

{'from': 17320,
 'gender': 17817,
 'perspective': 31409,
 'paulgaard': 30974,
 'points': 32035,
 'out': 30158,
 'that': 41221,
 'the': 41229,
 'labour': 23893,
 'markets': 25859,
 'of': 29487,
 'fishing': 16605,
 'villages': 44151,
 'have': 19212,
 'been': 5401,
 'highly': 19611,
 'segregated': 37317,
 'in': 20921,
 'terms': 41095,
 'existence': 15618,
 'male': 25572,
 'jobs': 22833,
 'and': 3469,
 'female': 16283,
 'however': 20021,
 'new': 28504,
 'business': 6873,
 'opportunities': 29841,
 'led': 24333,
 'to': 41630,
 'population': 32228,
 'peripheral': 31312,
 'areas': 4044,
 'now': 29057,
 'working': 45203,
 'service': 37557,
 'industry': 21264,
 'former': 17023,
 'boys': 6375,
 'girls': 18111,
 'are': 4040,
 'doing': 13104,
 'same': 36665,
 'indicates': 21161,
 'change': 7938,
 'because': 5370,
 'traditional': 41927,
 'boundaries': 6336,
 'between': 5660,
 'women': 45152,
 'men': 26434,
 'work': 45189,
 'being': 5451,
 'crossed': 10661,
 'but': 6886,
 'fact': 15924,
 'young': 455

 - with stop word removal

In [6]:
docs = text_df.text
count_vectorizer = CountVectorizer(ngram_range=(1,1),stop_words='english') 
count_vectorizer.fit(docs) 
len(count_vectorizer.vocabulary_)

45440

In [7]:
print(count_vectorizer.get_stop_words())

frozenset({'since', 'hers', 'always', 'thence', 'via', 'could', 'five', 'they', 'above', 'seemed', 'together', 'others', 'except', 'such', 'between', 'beside', 'below', 'around', 'afterwards', 'my', 'anyway', 'several', 'his', 'full', 'are', 'ever', 'our', 'de', 'eleven', 'becoming', 'onto', 'somewhere', 'done', 'made', 'every', 'show', 'we', 'from', 'while', 'thru', 'been', 'he', 'as', 'further', 'fifteen', 'so', 'meanwhile', 'next', 'those', 'with', 'though', 'third', 'inc', 'more', 'whenever', 'one', 'without', 'serious', 'yours', 'fill', 'never', 'mill', 'alone', 'were', 'but', 'sixty', 'move', 'then', 'same', 'the', 'nine', 'anyhow', 'least', 'eight', 'ten', 'thick', 'might', 'therefore', 'why', 'everything', 'not', 'hasnt', 'two', 'upon', 'already', 'someone', 'at', 'for', 'thin', 'over', 'must', 'forty', 'please', 'somehow', 'has', 'last', 'after', 'am', 'anything', 'sometimes', 'down', 'whither', 'cannot', 'ie', 'along', 'towards', 'nobody', 'have', 'con', 'would', 'everyone', 

 - difference is not the same as the stop_words length, why?
 - perhaps some stop words never appeared in our corpus

In [8]:
45738 - 45440, len(count_vectorizer.get_stop_words())


(298, 318)

In [9]:
count_vectorizer.vocabulary_.keys()

dict_keys(['gender', 'perspective', 'paulgaard', 'points', 'labour', 'markets', 'fishing', 'villages', 'highly', 'segregated', 'terms', 'existence', 'male', 'jobs', 'female', 'new', 'business', 'opportunities', 'led', 'population', 'peripheral', 'areas', 'working', 'service', 'industry', 'boys', 'girls', 'doing', 'indicates', 'change', 'traditional', 'boundaries', 'women', 'men', 'work', 'crossed', 'fact', 'young', 'people', 'represents', 'continuity', 'past', '2002', '102', 'refers', 'traditions', 'expectations', 'adults', 'participate', 'adult', 'culture', 'traditionally', 'actual', 'youth', 'described', 'earlier', '2015', 'concludes', 'norway', 'school', 'foreign', 'time', 'waster', 'stealing', 'instead', 'spend', 'considered', 'real', 'average', 'figure', 'masks', 'large', 'differences', 'regions', 'kazakhstan', 'number', 'annual', 'contacts', 'ranges', 'astana', 'mangystau', 'parts', 'likely', 'limited', 'access', 'primary', 'care', 'addition', 'poor', 'coverage', 'outpatient', 'p

 - take a look at a portion of the term-document matrix

In [10]:
docs = text_df.text
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer.fit(docs)
count_vector = count_vectorizer.transform(docs).toarray() 
count_vector_df = pd.DataFrame(count_vector, columns=count_vectorizer.get_feature_names_out())
term_freq = pd.DataFrame({"term": count_vector_df.columns.values, "freq" : count_vector_df.sum(axis=0)})
count_vector_df.loc[100:125,term_freq.sort_values(by="freq", ascending =False)[:20].term] # take a portion

,countries,women,development,health,water,public,social,education,policy,international,law,energy,national,rights,oecd,economic,use,new,level,income
100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
101,1,0,0,0,0,2,0,0,0,1,0,0,1,0,0,0,0,0,1,0
102,2,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0
103,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
104,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
105,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
106,1,3,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,2
107,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
108,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
109,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0


In [11]:
term_freq.sort_values(by="freq", ascending=False)

,term,freq
countries,countries,7761
women,women,5984
development,development,5312
health,health,4685
water,water,4664
...,...,...
envisioning,envisioning,1
envolee,envolee,1
ownerless,ownerless,1
envsec,envsec,1


In [12]:
term_freq.freq.sum()

1351051

In [13]:
docs.apply(lambda x: len(x.split())).sum()

2351777

 - stop word removal reduced document word count by 40%

 - check the bigrams

In [14]:
docs = text_df.text
count_vectorizer = CountVectorizer(ngram_range=(2,2), stop_words='english') 
count_vectorizer.fit(docs) 
len(count_vectorizer.vocabulary_)

834465

 - check the tri-grams

In [15]:
docs = text_df.text
count_vectorizer = CountVectorizer(ngram_range=(3,3), stop_words='english') 
count_vectorizer.fit(docs) 
len(count_vectorizer.vocabulary_)

1214215

In [16]:
# proportion of non-zeros
print('vocabulary size: ' , len(count_vectorizer.vocabulary_))
print('vector shape: ', count_vector.shape)
print("proportion of non-zeros: ", np.count_nonzero(count_vector)/(count_vector.shape[0]*count_vector.shape[1]))

vocabulary size:  1214215
vector shape:  (24669, 45440)
proportion of non-zeros:  0.0010100642720892218


In [17]:
# the lass line in this cell that calls for count_vector_df.sum will not finish 
# unless min_df is set to 2 or above in the CountVectorizer call at top of this cell, make sure to specify
count_vectorizer = CountVectorizer(ngram_range = (2,2),stop_words='english', min_df=5)
count_vector = count_vectorizer.fit_transform(docs).toarray()
count_vector_df = pd.DataFrame(count_vector, columns=count_vectorizer.get_feature_names_out())
bigram_freq = pd.DataFrame({"term": count_vector_df.columns.values, "freq" : count_vector_df.sum(axis=0)})

In [18]:
print(bigram_freq.query("term == 'rural areas'")["freq"])
print("vocabulary index is: ", count_vectorizer.vocabulary_.get("rural areas"))

rural areas    411
Name: freq, dtype: int64
vocabulary index is:  21012


In [19]:
print(count_vectorizer.vocabulary_[u'rural areas'])# u'string' for unicode string 'string'
print(count_vectorizer.vocabulary_.get(u'rural areas') )

21012
21012


### get count vector and then followed by sum is an expensive way to get word frequency
 - may be better rely on vocabulary, which is a dictionary

In [20]:
bigram_freq.sort_values(by="freq", ascending =False)[:30]

,term,freq
human rights,human rights,1981
climate change,climate change,1301
et al,et al,1167
oecd countries,oecd countries,898
health care,health care,881
international law,international law,782
united states,united states,778
developing countries,developing countries,772
long term,long term,743
labour market,labour market,720


In [21]:
count_vectorizer.vocabulary_

{'gender perspective': 9554,
 'labour markets': 13135,
 'gender segregated': 9563,
 'new business': 15837,
 'business opportunities': 2945,
 'peripheral areas': 17174,
 'boys girls': 2786,
 'women men': 25529,
 'men work': 14961,
 'young people': 25906,
 'people working': 17080,
 'young adults': 25899,
 'described earlier': 5848,
 'spend time': 22540,
 'average figure': 2262,
 'large differences': 13266,
 'parts population': 16900,
 'likely limited': 13885,
 'limited access': 13914,
 'access primary': 821,
 'primary care': 18313,
 'care addition': 3104,
 'addition poor': 1098,
 'findings consistent': 8923,
 'previous work': 18258,
 'differences wage': 6196,
 'wage inequality': 25070,
 'inequality countries': 12162,
 'fournier koske': 9267,
 'koske 2012': 13113,
 'returns education': 20637,
 'education important': 6995,
 'important role': 11377,
 'earnings inequality': 6716,
 '2014 countries': 431,
 'supply demand': 23184,
 'demand skills': 5742,
 'role played': 20966,
 'labour market':

In [22]:
bigram_freq.query("freq==5").nunique()


term    6038
freq       1
dtype: int64

### putting together as a function
 - enable stop words removal
 - enable unigram, bigram, tri-gram

In [23]:
def get_term_freq(docs, ngram_range = (1, 1), stop_words = None, min_df = 2): # min_df has to be 2 or larger
    count_vectorizer = CountVectorizer(ngram_range = ngram_range, stop_words = stop_words, min_df =min_df)
    count_vector = count_vectorizer.fit_transform(docs).toarray()
    count_vector_df = pd.DataFrame(count_vector, columns=count_vectorizer.get_feature_names_out())
    term_freq = pd.DataFrame({"term": count_vector_df.columns.values, "freq" : count_vector_df.sum(axis=0)})
    return term_freq

In [24]:
docs = text_df.text
term_freq_1_1_stop = get_term_freq(docs, stop_words='english').sort_values(by="freq", ascending=False)
term_freq_1_1_no_stop = get_term_freq(docs).sort_values(by="freq", ascending=False)
term_freq_2_2_stop = get_term_freq(docs, ngram_range = (2,2), stop_words = 'english').sort_values(by="freq", ascending=False)
term_freq_2_2_no_stop = get_term_freq(docs, ngram_range = (2,2)).sort_values(by="freq", ascending=False)

In [25]:
term_freq_2_2_no_stop.head(50)

,term,freq
of the,of the,18261
in the,in the,15030
to the,to the,6798
and the,and the,6567
on the,on the,5116
for the,for the,3901
to be,to be,3539
by the,by the,3214
such as,such as,3154
with the,with the,2847


In [26]:
term_freq_2_2_stop.head(50)

,term,freq
human rights,human rights,1981
climate change,climate change,1301
et al,et al,1167
oecd countries,oecd countries,898
health care,health care,881
international law,international law,782
united states,united states,778
developing countries,developing countries,772
long term,long term,743
labour market,labour market,720


In [36]:
term_freq_2_2_no_stop.shape, term_freq_2_2_stop.shape

((194394, 2), (148718, 2))

In [27]:
count_vectorizer.vocabulary_.items()

dict_items([('gender perspective', 9554), ('labour markets', 13135), ('gender segregated', 9563), ('new business', 15837), ('business opportunities', 2945), ('peripheral areas', 17174), ('boys girls', 2786), ('women men', 25529), ('men work', 14961), ('young people', 25906), ('people working', 17080), ('young adults', 25899), ('described earlier', 5848), ('spend time', 22540), ('average figure', 2262), ('large differences', 13266), ('parts population', 16900), ('likely limited', 13885), ('limited access', 13914), ('access primary', 821), ('primary care', 18313), ('care addition', 3104), ('addition poor', 1098), ('findings consistent', 8923), ('previous work', 18258), ('differences wage', 6196), ('wage inequality', 25070), ('inequality countries', 12162), ('fournier koske', 9267), ('koske 2012', 13113), ('returns education', 20637), ('education important', 6995), ('important role', 11377), ('earnings inequality', 6716), ('2014 countries', 431), ('supply demand', 23184), ('demand skills'

In [28]:
count_vectorizer.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [29]:
count_vectorizer.get_feature_names_out()

array(['000 additional', '000 ha', '000 hectares', ..., 'zero sum',
       'zero tolerance', 'zero waste'], dtype=object)

In [30]:
import re
t1 = text_df.text[1]
print(re.search(r'China', t1))
print(re.search(r'[a-z]\,',t1))


None
<re.Match object; span=(152, 154), match='u,'>


In [31]:
from nltk.corpus import wordnet
wordnet.synsets('dog'),wordnet.synsets('dog', pos=wordnet.VERB)

([Synset('dog.n.01'),
  Synset('frump.n.01'),
  Synset('dog.n.03'),
  Synset('cad.n.01'),
  Synset('frank.n.02'),
  Synset('pawl.n.01'),
  Synset('andiron.n.01'),
  Synset('chase.v.01')],
 [Synset('chase.v.01')])

In [32]:
synonyms = []
antonyms = []

for syn in wordnet.synsets("good"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

print(set(synonyms))
print(set(antonyms))

{'trade_good', 'goodness', 'well', 'adept', 'sound', 'upright', 'safe', 'soundly', 'proficient', 'unspoiled', 'unspoilt', 'secure', 'full', 'beneficial', 'estimable', 'good', 'in_effect', 'near', 'skilful', 'practiced', 'dear', 'undecomposed', 'thoroughly', 'right', 'skillful', 'effective', 'respectable', 'just', 'in_force', 'salutary', 'honest', 'commodity', 'expert', 'ripe', 'serious', 'honorable', 'dependable'}
{'bad', 'badness', 'evilness', 'evil', 'ill'}


In [33]:
syns = wordnet.synsets("good")

In [34]:
syns[0].examples()

['for your own good', "what's the good of worrying?"]

In [35]:
w1 = wordnet.synset('ship.n.01')
w2 = wordnet.synset('boat.n.01')
w3 = wordnet.synset('car.n.01')
w4 = wordnet.synset('cat.n.01')
print(w1.wup_similarity(w2)), print(w1.wup_similarity(w3)),print(w1.wup_similarity(w4)),print(w2.wup_similarity(w3)), print(w2.wup_similarity(w4)), print(w3.wup_similarity(w4))

0.9090909090909091
0.6956521739130435
0.32
0.6956521739130435
0.32
0.32


(None, None, None, None, None, None)